In [1]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM,Dense,Input,ConvLSTM2D,Reshape,Activation,Lambda,Softmax,Concatenate
from keras.layers import Dense, LSTM, Input, Embedding, TimeDistributed, Flatten, Dropout,RepeatVector,Reshape,Activation,Lambda,Softmax
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
import re
import pandas as pd
from keras import backend as K
import keras.backend as kerback
#from keras.models import Model
from tensorflow.keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow 

In [2]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy import random
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense

In [3]:
def prepare_data(f):
    data = pd.read_csv(f, sep="\n",header=None)
    list_listword=[]#liste des listes des mots de chaque tweet
    list_tweets=data[0].values.tolist()#liste des tweets
    l=[]
    for text in list_tweets:
        text = re.sub(r'http\S+', '', text)   # Remove URLs
        text = re.sub(r'—', ' ', text) 
        text = re.sub(r'@[a-zA-Z0-9_]+', '', text)  # Remove @ mentions
        text = text.strip(" ")   # Remove whitespace resulting from above
        text = re.sub(r' +', ' ', text)   # Remove redundant spaces
        l.append(text)
    
    return l

In [4]:
list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')

In [5]:
def tokenizer_data(list_tweets):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list_tweets)
    sequences = tokenizer.texts_to_sequences(list_tweets)
    vocab_size=len(tokenizer.word_index)
    return sequences, tokenizer, vocab_size

def max_tweet(sequences):
    max_length=0
    for i in range(len(sequences)):
        if(len(sequences[i])>max_length):
            max_length=len(sequences[i])
    return max_length

sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
max_length=max_tweet(sequences)
sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
y=sequences[:,-1]
print(y.shape)
y_train = y.reshape(-1, 1)
X=sequences
X_train = np.expand_dims(X, axis=2)
nb_tweet=len(sequences)
batch_size=max_length
print(vocab_size)

(66,)
841


In [6]:
 import keras.backend as kerback
# define the standalone generator model
def build_generator(n_outputs):#max_length nombre d'element das une tweet
   
    noise=Input(shape=(57,1))
    
    label=Input(shape=(57,))
    label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=2)([noise,label_emb])
    x1=LSTM(15, activation='relu',input_dim=n_outputs)(x0)
    #15 nombre de neurone ,batch_size=n_batch=20 
    # repeat vector
    x2=RepeatVector(n_outputs)(x1)
    # decoder layer
    x3=LSTM(15, activation='relu', return_sequences=True)(x2)
    x4=TimeDistributed(Dense(1))(x3)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x4)
    dense_outputs = [Dense(n_outputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)

    #generated_tweet=model(x0)
    model = Model([noise,label],merged)
    #model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('generator')
    print(model.summary())
    return  model

build_generator(57)

generator
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 57, 1)        841         input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 57, 2)        0           input_1[0][0]                    
                                                                 embedding[0]

In [7]:
# define the standalone discriminator model
def build_discriminator(n_inputs):
    tweet=Input(shape=(n_inputs,1)) #n_imput nombre des element 
    
    label=Input(shape=(n_inputs,))
    label_emb=Embedding(vocab_size,1)(label)
    
    x0=Concatenate(axis=1)([tweet,label_emb])
    x2=LSTM(15, activation='relu', input_dim=n_inputs)(x0)
    x3=RepeatVector(n_inputs)(x2)
    x4=LSTM(15, activation='relu', return_sequences=True)(x3)
    x5=TimeDistributed(Dense(1))(x4)
    unstacked = Lambda(lambda x: tensorflow.unstack(x, axis=2))(x5)
    dense_outputs = [Dense(n_inputs)(x) for x in unstacked]
    merged = Lambda(lambda x: K.stack(x, axis=2))(dense_outputs)
    x1=Flatten()(merged)
    x6=Dense(1, activation='sigmoid')(x1)
    model = Model([tweet,label], x6)
    print(model.summary())
    model.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])
    return model
build_discriminator(57)

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 57, 1)        841         input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 114, 1)       0           input_3[0][0]                    
                                                                 embedding_1[0][0]     

In [8]:
# define the combined generator and discriminator model
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    #model.add(generator)
    x0=Input(shape=(57,1))
    x1=Input(shape=(57,))
    x2=generator([x0,x1])
    print(x2[-1].shape)
    x3=discriminator([x2,x1])
    print(x3.shape)
    model = Model([x0,x1], x3)
    # add the discriminator
    #model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    print(model.summary())
    return model
define_gan(build_generator(57), build_discriminator(57))

generator
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 57, 1)        841         input_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 57, 2)        0           input_5[0][0]                    
                                                                 embedding_2[

In [9]:
def generate_real_samples():
    list_tweets=prepare_data('C:/Users/LENOVO/Desktop/PCD/tweet.txt')
    sequences, tokenizer, vocab_size=tokenizer_data(list_tweets)
    max_length=max_tweet(sequences)
    sequences=np.array([xi+[0]*(max_length-len(xi)) for xi in sequences])
    y=sequences[:,-1]
    #y_train = y.reshape(-1, 1)
    X=sequences
    X_train = np.expand_dims(X, axis=2)
    #print(X_train.shape)
    nb_tweet=len(sequences)
    batch_size=max_length
    
    return [X_train,y_train]
generate_real_samples()

[array([[[146],
         [  7],
         [ 46],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 87],
         [ 12],
         [  4],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 91],
         [ 59],
         [ 12],
         ...,
         [  0],
         [  0],
         [  0]],
 
        ...,
 
        [[ 68],
         [  3],
         [320],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[ 25],
         [  8],
         [ 24],
         ...,
         [  0],
         [  0],
         [  0]],
 
        [[834],
         [305],
         [  1],
         ...,
         [  6],
         [ 61],
         [ 62]]]),
 array([[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],

In [10]:
# Create batch labels to use when calling train_on_batch
valid = ones((max_length,1))
fake = zeros((max_length,1))

In [11]:
generator = build_generator(max_length)
# Build and compile the discriminator
discriminator = build_discriminator(max_length)
#discriminator.compile(optimizer='adam', loss='binary_crossentropy' ,metrics=['accuracy'])

# Build and compile the combined model
gan_model = define_gan(generator, discriminator)

generator
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 57)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 57, 1)]      0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 57, 1)        841         input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 57, 2)        0           input_11[0][0]                   
                                                                 embedding_4

In [12]:
def sample_tweet(epoch,noise,sampled_labels,g_model):
    result = g_model.predict([noise,sampled_labels])
    #print(result)
    #print(result.shape)
    #print(len(result))
    res=[]
    predicted_word =""
    for j in range(57):
        #predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == int(result[0][j]):
                predicted_word = predicted_word+' '+word
                break
    res.append(predicted_word)
    print(res)
    
    #for i in res:
    #    print(i)
    return res

In [13]:
def disc_text(model_disc, seed_text):
    #print(seed_text)
    sequences, tokenizer=tokenizer_data(seed_text)
    sequences= pad_sequences(sequences, maxlen=max_length, padding='post')
    y = model_disc.predict(sequences)
    #print(y)
    #for i in y:
    if(i<0.7):
        print("fake")
    else:
        print("real")

In [14]:
# Store the losses
d_losses = []
g_losses = []
sample_period = 100 # every `sample_period` steps generate and save some data"
Epoch=3000
#x_real= generate_real_samples()
######################## Main training loop###########################
for i in range(Epoch):
        ########################
        ###Train Discriminator##
        #######################
        # prepare real samples
        [X_train,y_train]= generate_real_samples()
        #print(X_train.shape)
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        #print(idx)
        tweets, labels = X_train[idx], y_train[idx]
        #print(tweets.shape)
        #print(labels.shape)
        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, 1))
        #print(noise.shape)
        # Generate fake tweets
        gen_twt= generator.predict([noise,labels])
        
        
        # Train the discriminator
        # both loss and accuracy are returned
        d_loss_real, d_acc_real=discriminator.train_on_batch([tweets,labels], valid)
        d_loss_fake, d_acc_fake =discriminator.train_on_batch([gen_twt,labels], fake)
        d_loss = 0.5 * (d_loss_real + d_loss_fake)
        d_acc  = 0.5 * (d_acc_real + d_acc_fake)
        
        #######################
        ### Train generator ###
        #######################
    
        # Condition on labels
        sampled_labels = np.random.randint(0, 10, batch_size)                          
        #print(sampled_labels.shape)                            
        y_gan = ones((57, 1))
        # update the generator via the discriminator's error
        noise=random.randn(57,1)
        #gan_model([noise,x_real[-1]], y_gan).summary()
        g_loss, n=gan_model.train_on_batch([noise,sampled_labels], y_gan)
        
        # Save the losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
        #if i % 2 == 100:#after 100 epoch we save
        #if i % sample_period == 0:
        print(f"epoch: {i+1}, d_loss: {d_loss:.2f}, \
        d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
        sample_tweet(i,noise,sampled_labels, generator)
        #disc_text(discriminator, tex)

epoch: 1, d_loss: 85.52,         d_acc: 0.36, g_loss: 0.69
['']
epoch: 2, d_loss: 0.38,         d_acc: 0.99, g_loss: 0.70
['']
epoch: 3, d_loss: 0.39,         d_acc: 0.93, g_loss: 0.71
['']
epoch: 4, d_loss: 21.73,         d_acc: 0.85, g_loss: 0.72
['']
epoch: 5, d_loss: 0.36,         d_acc: 0.96, g_loss: 0.73
['']
epoch: 6, d_loss: 0.37,         d_acc: 0.94, g_loss: 0.74
['']
epoch: 7, d_loss: 0.37,         d_acc: 0.93, g_loss: 0.75
['']
epoch: 8, d_loss: 0.43,         d_acc: 0.85, g_loss: 0.76
['']
epoch: 9, d_loss: 5.86,         d_acc: 0.92, g_loss: 0.77
['']
epoch: 10, d_loss: 0.39,         d_acc: 0.89, g_loss: 0.78
['']
epoch: 11, d_loss: 0.38,         d_acc: 0.89, g_loss: 0.80
['']
epoch: 12, d_loss: 0.44,         d_acc: 0.84, g_loss: 0.81
['']
epoch: 13, d_loss: 0.41,         d_acc: 0.85, g_loss: 0.82
['']
epoch: 14, d_loss: 0.45,         d_acc: 0.80, g_loss: 0.83
['']
epoch: 15, d_loss: 0.42,         d_acc: 0.82, g_loss: 0.84
['']
epoch: 16, d_loss: 2.38,         d_acc: 0.72, g

epoch: 78, d_loss: 2.26,         d_acc: 0.72, g_loss: 0.86
['']
epoch: 79, d_loss: 6.02,         d_acc: 0.69, g_loss: 0.86
['']
epoch: 80, d_loss: 4.74,         d_acc: 0.69, g_loss: 0.87
['']
epoch: 81, d_loss: 3.71,         d_acc: 0.73, g_loss: 0.87
['']
epoch: 82, d_loss: 4.02,         d_acc: 0.70, g_loss: 0.88
['']
epoch: 83, d_loss: 0.50,         d_acc: 0.74, g_loss: 0.89
['']
epoch: 84, d_loss: 0.45,         d_acc: 0.79, g_loss: 0.90
['']
epoch: 85, d_loss: 3.42,         d_acc: 0.74, g_loss: 0.90
['']
epoch: 86, d_loss: 0.51,         d_acc: 0.72, g_loss: 0.90
['']
epoch: 87, d_loss: 0.49,         d_acc: 0.74, g_loss: 0.91
['']
epoch: 88, d_loss: 0.50,         d_acc: 0.73, g_loss: 0.91
['']
epoch: 89, d_loss: 0.44,         d_acc: 0.79, g_loss: 0.91
['']
epoch: 90, d_loss: 2.55,         d_acc: 0.72, g_loss: 0.91
['']
epoch: 91, d_loss: 2.34,         d_acc: 0.65, g_loss: 0.91
['']
epoch: 92, d_loss: 0.51,         d_acc: 0.71, g_loss: 0.91
['']
epoch: 93, d_loss: 0.43,         d_acc: 

epoch: 205, d_loss: 0.40,         d_acc: 0.80, g_loss: 1.08
['']
epoch: 206, d_loss: 0.44,         d_acc: 0.76, g_loss: 1.08
['']
epoch: 207, d_loss: 0.44,         d_acc: 0.76, g_loss: 1.08
['']
epoch: 208, d_loss: 0.49,         d_acc: 0.71, g_loss: 1.08
['']
epoch: 209, d_loss: 0.41,         d_acc: 0.79, g_loss: 1.08
['']
epoch: 210, d_loss: 0.47,         d_acc: 0.73, g_loss: 1.08
['']
epoch: 211, d_loss: 0.44,         d_acc: 0.75, g_loss: 1.08
['']
epoch: 212, d_loss: 0.48,         d_acc: 0.72, g_loss: 1.08
['']
epoch: 213, d_loss: 0.48,         d_acc: 0.72, g_loss: 1.09
['']
epoch: 214, d_loss: 0.42,         d_acc: 0.78, g_loss: 1.09
['']
epoch: 215, d_loss: 0.47,         d_acc: 0.73, g_loss: 1.09
['']
epoch: 216, d_loss: 0.47,         d_acc: 0.73, g_loss: 1.09
['']
epoch: 217, d_loss: 0.45,         d_acc: 0.75, g_loss: 1.09
['']
epoch: 218, d_loss: 0.45,         d_acc: 0.75, g_loss: 1.09
['']
epoch: 219, d_loss: 0.46,         d_acc: 0.74, g_loss: 1.09
['']
epoch: 220, d_loss: 0.50,

epoch: 332, d_loss: 0.39,         d_acc: 0.79, g_loss: 1.16
['']
epoch: 333, d_loss: 0.46,         d_acc: 0.72, g_loss: 1.16
['']
epoch: 334, d_loss: 0.47,         d_acc: 0.71, g_loss: 1.16
['']
epoch: 335, d_loss: 0.43,         d_acc: 0.75, g_loss: 1.17
['']
epoch: 336, d_loss: 0.46,         d_acc: 0.72, g_loss: 1.17
['']
epoch: 337, d_loss: 0.45,         d_acc: 0.74, g_loss: 1.17
['']
epoch: 338, d_loss: 0.45,         d_acc: 0.74, g_loss: 1.17
['']
epoch: 339, d_loss: 0.43,         d_acc: 0.75, g_loss: 1.17
['']
epoch: 340, d_loss: 0.39,         d_acc: 0.80, g_loss: 1.17
['']
epoch: 341, d_loss: 0.40,         d_acc: 0.78, g_loss: 1.17
['']
epoch: 342, d_loss: 0.47,         d_acc: 0.71, g_loss: 1.17
['']
epoch: 343, d_loss: 0.47,         d_acc: 0.71, g_loss: 1.17
['']
epoch: 344, d_loss: 0.45,         d_acc: 0.73, g_loss: 1.17
['']
epoch: 345, d_loss: 0.44,         d_acc: 0.74, g_loss: 1.17
['']
epoch: 346, d_loss: 0.44,         d_acc: 0.74, g_loss: 1.17
['']
epoch: 347, d_loss: 0.47,

epoch: 459, d_loss: 0.38,         d_acc: 0.78, g_loss: 1.26
['']
epoch: 460, d_loss: 0.37,         d_acc: 0.78, g_loss: 1.26
['']
epoch: 461, d_loss: 0.44,         d_acc: 0.71, g_loss: 1.26
['']
epoch: 462, d_loss: 0.39,         d_acc: 0.76, g_loss: 1.26
['']
epoch: 463, d_loss: 0.46,         d_acc: 0.69, g_loss: 1.26
['']
epoch: 464, d_loss: 0.40,         d_acc: 0.75, g_loss: 1.27
['']
epoch: 465, d_loss: 0.37,         d_acc: 0.78, g_loss: 1.27
['']
epoch: 466, d_loss: 0.43,         d_acc: 0.72, g_loss: 1.27
['']
epoch: 467, d_loss: 0.42,         d_acc: 0.73, g_loss: 1.28
['']
epoch: 468, d_loss: 0.38,         d_acc: 0.77, g_loss: 1.29
['']
epoch: 469, d_loss: 0.43,         d_acc: 0.72, g_loss: 1.30
['']
epoch: 470, d_loss: 0.36,         d_acc: 0.79, g_loss: 1.33
['']
epoch: 471, d_loss: 0.40,         d_acc: 0.74, g_loss: 1.41
['']
epoch: 472, d_loss: 0.37,         d_acc: 0.75, g_loss: 1.64
['']
epoch: 473, d_loss: 0.36,         d_acc: 0.68, g_loss: 2.42
['']
epoch: 474, d_loss: 12.96

...

epoch: 2569, d_loss: 22.35,         d_acc: 0.99, g_loss: 3.26
['']
epoch: 2570, d_loss: 22.35,         d_acc: 0.99, g_loss: 3.26
[' to to to to to to in a in in in in a in of in and in and in and and the to to']
epoch: 2571, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.25
['']
epoch: 2572, d_loss: 0.03,         d_acc: 1.00, g_loss: 3.25
['']
epoch: 2573, d_loss: 44.67,         d_acc: 0.98, g_loss: 3.24
[' to to']
epoch: 2574, d_loss: 44.66,         d_acc: 0.98, g_loss: 3.25
[' to to and the and and and and and and and and the and the and the the to to']
epoch: 2575, d_loss: 0.03,         d_acc: 1.00, g_loss: 3.29
[' to']
epoch: 2576, d_loss: 0.03,         d_acc: 1.00, g_loss: 3.28
[' to to to to to to to to to to to to to to to to']
epoch: 2577, d_loss: 22.34,         d_acc: 0.99, g_loss: 3.26
[' to to to to to to to to to']
epoch: 2578, d_loss: 44.64,         d_acc: 0.98, g_loss: 3.28
[' to to to to to to a a in in in in a in in in and in and in and and the to to']
epoch: 2579, d_loss

epoch: 2659, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.41
[' to to to to to to to to to to to to to']
epoch: 2660, d_loss: 22.17,         d_acc: 0.99, g_loss: 3.37
['']
epoch: 2661, d_loss: 66.48,         d_acc: 0.97, g_loss: 3.40
['']
epoch: 2662, d_loss: 66.47,         d_acc: 0.97, g_loss: 3.39
[' to to the the the to to the the the to the to the to to to']
epoch: 2663, d_loss: 22.17,         d_acc: 0.99, g_loss: 3.39
[' to to to to to to to to']
epoch: 2664, d_loss: 22.18,         d_acc: 0.99, g_loss: 3.39
[' to to and the and and and and and and and and the and the and the the to to to']
epoch: 2665, d_loss: 66.50,         d_acc: 0.97, g_loss: 3.39
[' the to to to']
epoch: 2666, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.41
['']
epoch: 2667, d_loss: 22.16,         d_acc: 0.99, g_loss: 3.43
[' to to and the and and and and and and a and the and the and the the to to to']
epoch: 2668, d_loss: 22.15,         d_acc: 0.99, g_loss: 3.41
['']
epoch: 2669, d_loss: 0.02,         d_acc:

epoch: 2740, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.51
['']
epoch: 2741, d_loss: 22.01,         d_acc: 0.99, g_loss: 3.53
['']
epoch: 2742, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.51
['']
epoch: 2743, d_loss: 44.00,         d_acc: 0.98, g_loss: 3.50
[' to to to to to to to to to to to to to']
epoch: 2744, d_loss: 22.01,         d_acc: 0.99, g_loss: 3.51
['']
epoch: 2745, d_loss: 22.01,         d_acc: 0.99, g_loss: 3.52
[' to to to to to to a and in in a a a in in a and a the in and the the to to']
epoch: 2746, d_loss: 44.00,         d_acc: 0.98, g_loss: 3.53
['']
epoch: 2747, d_loss: 65.97,         d_acc: 0.97, g_loss: 3.53
['']
epoch: 2748, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.52
['']
epoch: 2749, d_loss: 44.02,         d_acc: 0.98, g_loss: 3.55
[' to to and the and and and and and and and and the and the and the the to to']
epoch: 2750, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.53
['']
epoch: 2751, d_loss: 0.02,         d_acc: 1.00, g_loss: 3.52
['']
epoch: 2752

epoch: 2832, d_loss: 21.79,         d_acc: 0.99, g_loss: 4.26
['']
epoch: 2833, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.60
[' to to to to to to to the the to to to to the to to']
epoch: 2834, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.64
[' to to']
epoch: 2835, d_loss: 43.56,         d_acc: 0.98, g_loss: 3.61
[' to to to']
epoch: 2836, d_loss: 21.78,         d_acc: 0.99, g_loss: 3.64
['']
epoch: 2837, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.63
[' to']
epoch: 2838, d_loss: 21.78,         d_acc: 0.99, g_loss: 3.62
[' to to to to to to to to to to to to to']
epoch: 2839, d_loss: 43.53,         d_acc: 0.98, g_loss: 3.62
['']
epoch: 2840, d_loss: 21.78,         d_acc: 0.99, g_loss: 3.65
[' the the the and the the the and and the the the to and to to to']
epoch: 2841, d_loss: 65.30,         d_acc: 0.97, g_loss: 3.62
['']
epoch: 2842, d_loss: 21.76,         d_acc: 0.99, g_loss: 3.65
[' to the the and and and the the and and and the and to and to to to to']
epoch: 2843, d_loss: 21.

epoch: 2922, d_loss: 21.59,         d_acc: 0.99, g_loss: 3.74
[' to to to to and and a a a and and a a a the a the a the the to to to']
epoch: 2923, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.72
['']
epoch: 2924, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.71
[' the the and and the the the and and the the the to and to to to']
epoch: 2925, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.76
['']
epoch: 2926, d_loss: 21.59,         d_acc: 0.99, g_loss: 3.75
['']
epoch: 2927, d_loss: 43.15,         d_acc: 0.98, g_loss: 3.73
[' to to to to to to to to to']
epoch: 2928, d_loss: 21.57,         d_acc: 0.99, g_loss: 3.72
['']
epoch: 2929, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.74
['']
epoch: 2930, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.75
['']
epoch: 2931, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.74
[' to']
epoch: 2932, d_loss: 43.22,         d_acc: 0.98, g_loss: 3.74
['']
epoch: 2933, d_loss: 0.01,         d_acc: 1.00, g_loss: 3.77
['']
epoch: 2934, d_loss: 21.56,         d_ac